# Init

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
dir_origin = '/content/drive/MyDrive/Smartech/MinaRumas/'
os.chdir(dir_origin)
%pwd
%ls


models/  Process_Video.ipynb  video/


In [3]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)


In [4]:
import os
from datetime import datetime

def generar_ruta_salida(video_path):
    """
    Genera una ruta de salida para un video con el formato:
    misma carpeta que el video original, en una subcarpeta 'output/{nombre_input}/',
    y nombre: output_{nombre_input}_{fecha_hora}.mp4
    """
    folder, filename = os.path.split(video_path)
    name, ext = os.path.splitext(filename)
    fecha_hora = datetime.now().strftime('%Y%m%d_%H%M')
    output_filename = f'output_{name}_{fecha_hora}.mp4'

    # Carpeta de salida: misma carpeta + subcarpeta 'output/{nombre_input}'
    output_folder = os.path.join(folder, 'output', name)

    # Crear carpeta si no existe
    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, output_filename)
    return output_path

# video_path = 'videos/video_cam2.mp4'
# output_path = generar_ruta_salida(video_path)


# Ruma logic

## import

In [5]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import time
import json
from datetime import datetime
import os
from collections import defaultdict, deque

## Logic class

In [6]:
class RumaMonitor:
    def __init__(self, model_det_path, model_seg_path, detection_zone, camera_sn):
        """
        Inicializa el monitor de rumas

        Args:
            model_det_path: Ruta al modelo de detección
            model_seg_path: Ruta al modelo de segmentación
            detection_zone: Polígono que define la zona de detección
            camera_sn: Número de serie de la cámara
        """
        self.model_det = YOLO(model_det_path)
        self.model_seg = YOLO(model_seg_path)
        self.detection_zone = detection_zone
        self.camera_sn = camera_sn

        # Tracking de rumas
        self.rumas = {}  # ruma_id: RumaData
        self.next_ruma_id = 1
        self.candidate_rumas = {}  # Para validar nuevas rumas por 10 frames

        # Estados de alertas
        self.object_in_zone = False
        self.object_interacting = False
        self.ruma_variation = False
        self.last_interaction_frame = {}  # Para controlar cuando mostrar porcentajes

        # Configuración de colores
        self.RUMA_COLOR = (0, 255, 0)
        self.PERSON_COLOR = (255, 255, 0)
        self.VEHICLE_COLOR = (0, 0, 255)
        self.TEXT_COLOR_WHITE = (255, 255, 255)
        self.TEXT_COLOR_GREEN = (0, 255, 0)
        self.TEXT_COLOR_RED = (0, 0, 255)

        # Crear carpeta de alertas
        self.setup_alerts_folder()

    def setup_alerts_folder(self):
        """Crea la estructura de carpetas para alertas"""
        self.alerts_base_path = Path("alerts")
        self.alerts_base_path.mkdir(exist_ok=True)

        today = datetime.now().strftime("%Y-%m-%d")
        self.today_alerts_path = self.alerts_base_path / today
        self.today_alerts_path.mkdir(exist_ok=True)

    class RumaData:
        def __init__(self, ruma_id, initial_mask, initial_area, centroid):
            self.id = ruma_id
            self.initial_mask = initial_mask
            self.initial_area = initial_area
            self.current_area = initial_area
            self.centroid = centroid
            self.percentage = 100.0
            self.last_seen_frame = 0
            self.is_active = True
            self.frames_without_interaction = 0
            self.last_stable_percentage = 100.0
            self.label_position = (centroid[0] - 30, centroid[1] - 10)  # Posición fija del label

    def is_point_in_polygon(self, point, polygon):
        """Verifica si un punto está dentro de un polígono usando el algoritmo de Ray Casting"""
        x, y = point
        n = len(polygon)
        inside = False

        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y

        return inside

    def calculate_intersection(self, box, mask_points):
        """Calcula si hay intersección entre un bounding box y una máscara"""
        x1, y1, x2, y2 = box

        for point in mask_points:
            px, py = point
            if x1 <= px <= x2 and y1 <= py <= y2:
                return True

        corners = [(x1, y1), (x1, y2), (x2, y1), (x2, y2)]
        for corner in corners:
            if self.is_point_in_polygon(corner, mask_points):
                return True

        return False

    def put_text_with_background(self, img, text, position, font=cv2.FONT_HERSHEY_SIMPLEX,
                                font_scale=0.4, color=(255,255,255), thickness=1,
                                bg_color=(0,0,0), bg_alpha=0.6):
        """Coloca texto con fondo semitransparente para mejor legibilidad"""
        (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)

        x, y = position
        bg_img = img.copy()
        padding = 5

        cv2.rectangle(bg_img, (x-padding, y-text_height-padding),
                     (x+text_width+padding, y+padding), bg_color, -1)

        overlay = cv2.addWeighted(bg_img, bg_alpha, img, 1-bg_alpha, 0)
        cv2.putText(overlay, text, (x, y), font, font_scale, color, thickness)

        return overlay

    def find_closest_ruma(self, centroid, max_distance=50): # max_distance=100
        """Encuentra la ruma más cercana a un centroide dado"""
        min_distance = float('inf')
        closest_ruma = None

        for ruma_id, ruma in self.rumas.items():
            if not ruma.is_active:
                continue

            dist = ((centroid[0] - ruma.centroid[0])**2 +
                   (centroid[1] - ruma.centroid[1])**2)**0.5

            if dist < min_distance and dist < max_distance:
                min_distance = dist
                closest_ruma = ruma_id

        return closest_ruma, min_distance

    def add_candidate_ruma(self, mask, centroid, frame_count):
        """Agrega una candidata a ruma para validación"""
        candidate_key = f"candidate_{centroid[0]}_{centroid[1]}"

        if candidate_key not in self.candidate_rumas:
            self.candidate_rumas[candidate_key] = {
                'mask': mask,
                'centroid': centroid,
                'area': cv2.contourArea(mask.astype(np.int32)),
                'first_frame': frame_count,
                'confirmations': 1
            }
        else:
            self.candidate_rumas[candidate_key]['confirmations'] += 1

            # Si se confirma por 10 frames, crear nueva ruma
            if self.candidate_rumas[candidate_key]['confirmations'] >= 6: # antes 10
                new_ruma = self.RumaData(
                    self.next_ruma_id,
                    mask,
                    cv2.contourArea(mask.astype(np.int32)),
                    centroid
                )
                self.rumas[self.next_ruma_id] = new_ruma
                print(f"Nueva ruma creada: ID {self.next_ruma_id}")
                self.next_ruma_id += 1
                del self.candidate_rumas[candidate_key]

    def update_ruma(self, ruma_id, mask, frame_count):
        """Actualiza los datos de una ruma existente"""
        ruma = self.rumas[ruma_id]
        ruma.current_area = cv2.contourArea(mask.astype(np.int32))
        ruma.percentage = (ruma.current_area / ruma.initial_area) * 100
        ruma.last_seen_frame = frame_count

        # Actualizar centroide
        ruma.centroid = (int(np.mean([p[0] for p in mask])),
                        int(np.mean([p[1] for p in mask])))

        # Si la ruma llega a 10% o menos, marcarla como inactiva
        if ruma.percentage <= 10:
            ruma.is_active = False
            print(f"Ruma {ruma_id} eliminada (porcentaje: {ruma.percentage:.1f}%)")

    def save_alert(self, alert_type, frame_with_drawings, frame_count, fps):
        """Guarda una alerta en JSON y el frame correspondiente"""
        timestamp = datetime.now()

        # Crear carpeta de la fecha si no existe
        date_folder = self.alerts_base_path / timestamp.strftime("%Y-%m-%d")
        date_folder.mkdir(exist_ok=True)

        # Calcular tiempo del video
        video_time_seconds = frame_count / fps

        # Metadata de la alerta
        alert_data = {
            "cameraSN": self.camera_sn,
            "alert_type": alert_type,
            "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "video_time_seconds": video_time_seconds, # Eliminar
            "frame_number": frame_count # Eliminar
        }

        # Nombres de archivos
        base_filename = f"{timestamp.strftime('%H-%M-%S')}_{alert_type}_{frame_count}"
        json_filename = f"{base_filename}.json"
        image_filename = f"{base_filename}.jpg"

        # Guardar JSON
        json_path = date_folder / json_filename
        with open(json_path, 'w') as f:
            json.dump(alert_data, f, indent=2)

        # Guardar imagen
        image_path = date_folder / image_filename
        cv2.imwrite(str(image_path), frame_with_drawings)

        print(f"Alerta guardada: {alert_type} - {timestamp.strftime('%H:%M:%S')}")

    def process_detections(self, frame, frame_count):
        """Procesa las detecciones de personas y vehículos"""
        rumas_interacting = set()
        object_in_zone = False

        result_det = self.model_det.track(frame, conf=0.5, persist=True, verbose=False)

        if (result_det is not None) and len(result_det) > 0:
            boxes = result_det[0].boxes

            if (boxes is not None) and len(boxes) > 0:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])

                    if conf > 0.5:
                        color = self.PERSON_COLOR if cls == 0 else self.VEHICLE_COLOR
                        label = 'persona' if cls == 0 else 'maquinaria'

                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        frame = self.put_text_with_background(
                            frame, label, (x1, y1 - 5),
                            color=self.TEXT_COLOR_WHITE, font_scale=0.6
                        )

                        # Verificar si está en la zona
                        center_x = (x1 + x2) // 2
                        center_y = (y1 + y2) // 2

                        if self.is_point_in_polygon((center_x, center_y), self.detection_zone):
                            object_in_zone = True

                        # Verificar interacción con rumas
                        for ruma_id, ruma in self.rumas.items():
                            if not ruma.is_active:
                                continue
                            if self.calculate_intersection([x1, y1, x2, y2], ruma.initial_mask):
                                rumas_interacting.add(ruma_id)
                                self.last_interaction_frame[ruma_id] = frame_count

        return frame, object_in_zone, rumas_interacting

    def process_segmentation(self, frame, frame_count, rumas_interacting):
        """Procesa la segmentación de rumas"""
        result_seg = self.model_seg(frame, conf=0.5, verbose=False)
        ruma_variation = False

        if result_seg and len(result_seg) > 0:
            for r in result_seg:
                if r.masks is not None:
                    masks = r.masks.xy

                    for mask in masks:
                        centroid_x = int(np.mean([p[0] for p in mask]))
                        centroid_y = int(np.mean([p[1] for p in mask]))
                        centroid = (centroid_x, centroid_y)

                        # Solo procesar rumas dentro de la zona de detección
                        if not self.is_point_in_polygon(centroid, self.detection_zone):
                            continue

                        # Buscar ruma existente más cercana
                        closest_ruma_id, distance = self.find_closest_ruma(centroid)

                        if closest_ruma_id is not None:
                            # Actualizar ruma existente
                            self.update_ruma(closest_ruma_id, mask, frame_count)
                            ruma = self.rumas[closest_ruma_id]

                            # Dibujar la ruma
                            overlay = frame.copy()
                            cv2.fillPoly(overlay, [mask.astype(np.int32)], self.RUMA_COLOR)
                            frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)

                            # --------  Lógica mejorada para mostrar porcentaje --------- #
                            is_interacting = closest_ruma_id in rumas_interacting
                            if is_interacting:
                                ruma.frames_without_interaction = 0
                                ruma.last_stable_percentage = ruma.percentage  # Reiniciar al interactuar
                            else:
                                ruma.frames_without_interaction += 1
                                # Mientras no se llega a 30 frames, guarda el mayor porcentaje
                                if ruma.frames_without_interaction < 15: #30
                                    ruma.last_stable_percentage = max(ruma.last_stable_percentage, ruma.percentage)

                            # Limita todo a máximo 100%
                            ruma.percentage = min(ruma.percentage, 100)
                            ruma.last_stable_percentage = min(ruma.last_stable_percentage, 100)

                            # Decide qué mostrar
                            if ruma.frames_without_interaction >= 30:
                                display_percentage = ruma.last_stable_percentage
                            else:
                                display_percentage = ruma.percentage

                            #--------                                          -------#

                            # Detectar variación significativa
                            if display_percentage < 95:
                                ruma_variation = True

                            # Mostrar ID y porcentaje usando posición fija del label
                            label_text = f"R{ruma.id} | {display_percentage:.1f}%"
                            frame = self.put_text_with_background(
                                frame, label_text, ruma.label_position,  # ← Usar posición fija
                                font_scale=0.6, color=self.TEXT_COLOR_WHITE
                            )

                        else:
                            # Posible nueva ruma - agregar como candidata
                            self.add_candidate_ruma(mask, centroid, frame_count)

        # Limpiar candidatas antiguas (más de 100 frames sin confirmación)
        to_remove = []
        for key, candidate in self.candidate_rumas.items():
            if frame_count - candidate['first_frame'] > 100:
                to_remove.append(key)

        for key in to_remove:
            del self.candidate_rumas[key]

        return frame, ruma_variation

    def draw_zone_and_status(self, frame, object_in_zone, object_interacting, ruma_variation):
        """Dibuja la zona de detección y el estado de las alertas"""
        width = frame.shape[1]

        # Dibujar zona de detección
        pts = self.detection_zone.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts], True, (0, 255, 255), 2, lineType=cv2.LINE_AA)

        # Textos de estado
        text_y_start = 50

        zone_text = "Movimiento en la zona" if object_in_zone else "Zona despejada"
        zone_color = self.TEXT_COLOR_RED if object_in_zone else self.TEXT_COLOR_GREEN
        frame = self.put_text_with_background(
            frame, zone_text, (width - 650, text_y_start),
            color=zone_color, font_scale=1.5
        )

        interact_text = "Interaccion con las rumas" if object_interacting else "Sin interacciones"
        interact_color = self.TEXT_COLOR_RED if object_interacting else self.TEXT_COLOR_GREEN
        frame = self.put_text_with_background(
            frame, interact_text, (width - 650, text_y_start + 60),
            color=interact_color, font_scale=1.5
        )

        variation_text = "Variacion en las rumas" if ruma_variation else "Rumas en reposo"
        variation_color = self.TEXT_COLOR_RED if ruma_variation else self.TEXT_COLOR_GREEN
        frame = self.put_text_with_background(
            frame, variation_text, (width - 650, text_y_start + 120),
            color=variation_color, font_scale=1.5
        )

        return frame

    def process_frame(self, frame, frame_count, fps):
        """Procesa un frame completo"""
        frame_with_drawings = frame.copy()

        # Procesar detecciones
        frame_with_drawings, object_in_zone, rumas_interacting = self.process_detections(
            frame_with_drawings, frame_count
        )

        # Procesar segmentación
        frame_with_drawings, ruma_variation = self.process_segmentation(
            frame_with_drawings, frame_count, rumas_interacting
        )

        # Determinar estados de alerta
        object_interacting = len(rumas_interacting) > 0

        # Dibujar zona y estado
        frame_with_drawings = self.draw_zone_and_status(
            frame_with_drawings, object_in_zone, object_interacting, ruma_variation
        )

        # Guardar alertas si hay cambios de estado
        current_alerts = {
            'movement': object_in_zone,
            'interaction': object_interacting,
            'variation': ruma_variation
        }

        previous_alerts = {
            'movement': self.object_in_zone,
            'interaction': self.object_interacting,
            'variation': self.ruma_variation
        }

        # Guardar alertas solo cuando se activan (cambio de False a True)
        for alert_type, current_state in current_alerts.items():
            if current_state and not previous_alerts[alert_type]:
                alert_names = {
                    'movement': 'movimiento_zona',
                    'interaction': 'interaccion_rumas',
                    'variation': 'variacion_rumas'
                }
                self.save_alert(alert_names[alert_type], frame_with_drawings, frame_count, fps)

        # Actualizar estados
        self.object_in_zone = object_in_zone
        self.object_interacting = object_interacting
        self.ruma_variation = ruma_variation

        return frame_with_drawings

# Process video

In [7]:
def process_video(video_path, output_path, start_time_sec, end_time_sec,
                 model_det_path, model_seg_path, detection_zone, camera_number):
    """
    Procesa un video completo usando el monitor de rumas

    Args:
        video_path: Ruta del video de entrada
        output_path: Ruta del video de salida
        start_time_sec: Tiempo de inicio en segundos
        end_time_sec: Tiempo de fin en segundos
        model_det_path: Ruta del modelo de detección
        model_seg_path: Ruta del modelo de segmentación
        detection_zone: Zona de detección como array numpy
        camera_number: Número de cámara (1, 2, o 3)
    """

    # Mapear número de cámara a serial
    camera_serials = {
        1: "DS-7104NI-Q1-1",
        2: "DS-7104NI-Q1-2",
        3: "DS-7104NI-Q1-3"
    }

    camera_sn = camera_serials.get(camera_number, "DS-7104NI-Q1-1")

    # Inicializar monitor
    monitor = RumaMonitor(model_det_path, model_seg_path, detection_zone, camera_sn)

    # Configurar video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video: {width}x{height} @ {fps} FPS")

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    start_frame = int(start_time_sec * fps)
    end_frame = int(end_time_sec * fps)

    print(f"Procesando frames {start_frame} a {end_frame}")

    frame_count = 0

    with torch.no_grad():
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count > end_frame:
                break

            if frame_count >= start_frame:
                # Procesar frame
                processed_frame = monitor.process_frame(frame, frame_count, fps)
                out.write(processed_frame)

                # Progreso
                if frame_count % 50 == 0:
                    print(f"Procesados {frame_count} frames")
                    print(f"Rumas activas: {sum(1 for r in monitor.rumas.values() if r.is_active)}")

            frame_count += 1

    cap.release()
    out.release()
    print(f"Procesamiento completado. Video guardado en {output_path}")
    print(f"Total de rumas detectadas: {len(monitor.rumas)}")

# Uso de lógica

In [8]:
# Ejemplo de uso
if __name__ == "__main__":
    video_path = 'video/video_demo_3.mp4'
    output_path = generar_ruta_salida(video_path)
    start_time_sec = 15
    end_time_sec = 50

    model_det_path = 'models/model_detection.pt'
    model_seg_path = 'models/model_segmentation.pt'

    detection_zone = np.array([
        [20, 691], [18, 779], [414, 881], [709, 759],
        [1675, 1060], [1902, 890], [1902, 667], [704, 416]
    ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)

    camera_number = 1  # Cambiar según la cámara

    process_video(
        video_path, output_path, start_time_sec, end_time_sec,
        model_det_path, model_seg_path, detection_zone, camera_number
    )


Video: 1920x1080 @ 30.001940617116244 FPS
Procesando frames 450 a 1500
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Alerta guardada: movimiento_zona - 18:36:13
Procesados 450 frames
Rumas activas: 0
Nueva ruma creada: ID 1
Nueva ruma creada: ID 2
Nueva ruma creada: ID 3
Nueva ruma creada: ID 4
Nueva ruma creada: ID 5
Nueva ruma creada: ID 6
Nueva ruma creada: ID 7
Nueva ruma creada: ID 8
Alerta guardada: interaccion_rumas - 18:36:19
Alerta guardada: variacion_rumas - 18:36:19
Procesados 500 frames
Rumas activas: 8
Nueva ruma creada: ID 9
Alerta guardada: variacion_rumas - 18:36:23
Alerta guardada: variacion_rumas - 18:36:29
Nueva ruma creada: ID 10
Nueva ruma creada: ID 11
Alerta guardada: variacion_rumas - 18:36:39
Procesados 550 frames
Rumas activas: 11
Alerta guardada: variacion_rumas - 18:36:43
Alerta guardada: v